In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

## 1. Loading both train and test data

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv("../input/train.tsv",sep = "\t")

In [ ]:
train.head()

In [ ]:
test = pd.read_csv("../input/test.tsv",sep = "\t")

In [ ]:
test.head()

In [ ]:
train["Sentiment"].unique()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.isnull().sum(axis=0)

In [ ]:
test.isnull().sum(axis = 0)

Observed that there are multiple phrases corresponding to a single sentence ID

In [ ]:
train["SentenceId"].value_counts()[0:5]

In [ ]:
test["SentenceId"].value_counts()[0:5]

Similar pattern observed in both train and test datasets

In [ ]:
len(train["SentenceId"].unique())+len(test["SentenceId"].unique())

In [ ]:
len(train["PhraseId"].unique())+len(test["PhraseId"].unique())

https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf checked to seee if the count was matching with this paper.

## 2. Creating the weights using TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(analyzer = "word", stop_words = 'english', min_df=0.01,max_df=0.9,ngram_range = (1,3))

In [ ]:
tfidf.fit(train["Phrase"])

In [ ]:
train_tfidf = tfidf.transform(train["Phrase"])

In [ ]:
X_train = train_tfidf.todense()

In [ ]:
Y_train = train["Sentiment"]

## Building the model

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
NB = MultinomialNB()

In [ ]:
NB.fit(X_train,Y_train)

In [ ]:
test_tfidf = tfidf.transform(test["Phrase"])

In [ ]:
x_test = test_tfidf.todense()

In [ ]:
x_test.shape

In [ ]:
y_pred = NB.predict(x_test)

In [ ]:
type(y_pred)

In [ ]:
y_pred_df = pd.DataFrame(y_pred, columns = ["Sentiment"])

In [ ]:
y_pred_df.head()

In [ ]:
sub = pd.concat([test["PhraseId"],y_pred_df],axis = 1)

In [ ]:
sub.head()

In [ ]:
#sub.to_csv('submission.csv', index=False)

### Trying with Count vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

In [ ]:
pattern = RegexpTokenizer(r'[a-zA-Z0-9]+')

In [ ]:
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = pattern.tokenize)

In [ ]:
cv.fit(train['Phrase'])

In [ ]:
train_cv = cv.transform(train["Phrase"])

In [ ]:
train_cv

In [ ]:
X_train2 = train_cv.todense()

In [ ]:
Y_train2 = train["Sentiment"]

In [ ]:
from sklearn.linear_model import SGDClassifier
sv = SGDClassifier()

In [ ]:
##sv.fit(X_train2,Y_train2)

Kernel is dead when we try to fit NB/SVm in case of Count vectorization. The matrix is very sparse.

### try tfidf for SVM

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sv = SGDClassifier(max_iter = 200)

In [ ]:
sv.fit(X_train,Y_train)

In [ ]:
y_pred2 = sv.predict(x_test)

In [ ]:
y_pred2_df = pd.DataFrame(y_pred2, columns = ["Sentiment"])

In [ ]:
sub2 = pd.concat([test["PhraseId"],y_pred2_df],axis = 1)

In [ ]:
sub2.head()

In [ ]:
#sub2.to_csv('submission2.csv', index=False)

### Trying with Keras

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
X_train = train['Phrase']

In [ ]:
train.dtypes

In [ ]:
from keras.utils import to_categorical

In [ ]:
Y_train = to_categorical(train['Sentiment'].values)

In [ ]:
Y_train.shape

In [ ]:
tz = Tokenizer(num_words = 10000, lower = True)

In [ ]:
tz.fit_on_texts(list(X_train))

In [ ]:
X_train2 = tz.texts_to_sequences(X_train)

In [ ]:
type(X_train2)

In [ ]:
len(X_train2)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X_train2 = pad_sequences(X_train2, maxlen=100)

In [ ]:
X_train2.shape

In [ ]:
X_test2 = tz.texts_to_sequences(test["Phrase"])

In [ ]:
X_test2 = pad_sequences(X_test2, maxlen=100)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train2, Y_train, test_size=0.20)

In [ ]:
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Embedding(10000,100,mask_zero=True))
model.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train,validation_data = (X_val,Y_val), epochs = 4, batch_size = 32)

In [ ]:
sub2['Sentiment'] = model.predict_classes(X_test2, batch_size=32, verbose=1)

In [ ]:
sub2.head()

In [ ]:
sub2.to_csv('submission3.csv', index=False)